In [1]:
import numpy as np
from scipy.stats import binned_statistic
from tqdm import tqdm, trange
import seaborn
import matplotlib.pyplot as plt
import os
import sys


# box = sys.argv[1]
box = 'Box_n50_0_1400'
curr_run_fname = '/oak/stanford/orgs/kipac/aemulus/aemulus_nu/%s/'%(box)
rockstar_dir = curr_run_fname+'output/rockstar/'

f = open(rockstar_dir+'savelist.txt', 'r')
savelist = f.read().split()
f.close()

N_snapshots = len(savelist)

i=0

import pickle

NvMs = {}

NvM_fname = '/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/'+curr_run_fname.split('/')[-2]+'_NvsM.pkl'
NvM_f = open(NvM_fname, 'rb')
NvMs = pickle.load(NvM_f)
NvM_f.close()

In [2]:
VOL = NvMs[0.25]['vol']

In [3]:
# jackknife = {}

f_pos = open('/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/'+box+'_pos', 'r')

tot_bin_idx = []
tot_N = []
tot_pos = []
offsets = {}


for a in tqdm(NvMs):
    snapshot_pos  = f_pos.readline().strip().split(',')
    snapshot_pos  = [np.array(pos.split(), dtype=np.float32) for pos in snapshot_pos if pos != '']
    snapshot_pos  = np.array(snapshot_pos)

    offsets[a] = len(tot_N)
    tot_N += [n for n in NvMs[a]['N']]
    tot_bin_idx += [bi+offsets[a]-1 for bi in NvMs[a]['bin_idx'] if bi != 0] #if bi=0 then mass below min mass threshold
    tot_pos += [[a, x, y, z] for (i,(x,y,z)) in enumerate(snapshot_pos) if NvMs[a]['bin_idx'][i] != 0]
    assert(len(snapshot_pos) == len(NvMs[a]['bin_idx']))
    assert(len(tot_pos) == len(tot_bin_idx))
f_pos.close()



100%|██████████| 16/16 [07:29<00:00, 28.09s/it]


In [ ]:
print(np.shape(tot_pos))

In [ ]:
import numpy as np
from scipy.spatial import cKDTree

# Create a k-D tree for spatial subdivision
tree = cKDTree(tot_pos)

In [ ]:
# Define the parameters
TIME_RANGE = (0.25, 1.0)
NUM_SUBREGIONS = 2e8
num_particles = np.shape(tot_pos)[0]
print(num_particles)

In [ ]:
subregion_size = num_particles // NUM_SUBREGIONS
subregion_indices = np.array_split(np.arange(num_particles), NUM_SUBREGIONS)
subregions = [tot_pos[idx] for idx in subregion_indices]

In [ ]:
# Calculate the mean histogram
mean_histogram = np.mean(histograms, axis=0)

# Initialize the covariance matrix
covariance = np.zeros((len(a_values) * num_bins, len(a_values) * num_bins))

In [ ]:
# Compute the mass histograms for each spatial region
mass_histograms = np.zeros((n_regions, n_time_bins))
N_subsamples = 2e10

for i in trange(N_subsamples):
    curr_N = np.zeros_like(tot_N)
    start_idx = i * sample_size
    end_idx = start_idx + sample_size
    if i == N_subsamples - 1:
        end_idx = len(shuffled)  # For the last subsample, adjust end index to include remaining points
    for halo in shuffled[start_idx:end_idx]:
        curr_N[halo] += 1
    #get the number count of halos in the mass bins in this subcube
    bin_counts += [curr_N]

for i in range(N_subsamples):
    region_particles = kdtree.query_ball_point([region_coords[i], region_coords[i], region_coords[i]], region_size)
    region_masses = masses[region_particles]
    mass_histograms[i] = np.histogram(time_coords[region_particles], bins=time_bins, weights=region_masses)[0]

In [ ]:
# Initialize arrays for mass histograms and mean histogram
histograms = np.zeros((tree.n_leaves, len(a_values), num_bins))
mean_histogram = np.zeros((len(a_values), num_bins))

# Loop over all subspaces
for i, leaf in enumerate(tree.query_ball_tree(tree, subspace_size)):

    # Get the particles in the current subspace
    subspace_particles = space[leaf]

    # Calculate the mass in each bin for each `a` value
    for j, a_val in enumerate(a_values):
        # Get the particles with the current `a` value
        a_particles = subspace_particles[subspace_particles[:, 0] == a_val]
        # Calculate the bin indices for each particle
        bin_indices = np.floor(num_bins * a_particles[:, 1:]).astype(int)
        # Sum the mass in each bin
        for k in range(num_bins):
            histograms[i, j, k] = np.sum(a_particles[bin_indices == k, 0])

    # Calculate the mean histogram over all subspaces
    mean_histogram += histograms[i] / tree.n_leaves

# Initialize array for covariance matrix
covariance = np.zeros((len(a_values) * num_bins, len(a_values) * num_bins))

# Loop over all subspaces and calculate the mean histogram without that subspace
for i, leaf in enumerate(tree.query_ball_tree(tree, subspace_size)):
    # Get the mass histograms for all other subspaces
    other_histograms = histograms[np.setdiff1d(np.arange(tree.n_leaves), i)]
    # Calculate the mean histogram without the current subspace
    other_mean_histogram = np.sum(other_histograms, axis=0) / (tree.n_leaves - 1)
    # Calculate the contribution of the current subs


In [ ]:
jackknife_covs_fname = '/oak/stanford/orgs/kipac/users/delon/aemulusnu_massfunction/'+curr_run_fname.split('/')[-2]+'_jackknife_covs.pkl'
jackknife_covs_f = open(jackknife_covs_fname, 'wb')
pickle.dump(jackknife, jackknife_covs_f)
jackknife_covs_f.close()